<a href="https://colab.research.google.com/github/anaguilarar/ElementsConcentration/blob/main/example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/anaguilarar/ElementsConcentration.git


Cloning into 'ElementsConcentration'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 37 (delta 10), reused 34 (delta 10), pack-reused 0
Unpacking objects: 100% (37/37), done.


In [2]:
!pip install -U scikit-multiflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 38.3 MB/s eta 0:00:00


## Elements concentration prediction

This repositery aimed at estimting elements concentration using multi-target regression. 

In [3]:
### Multitarget prediction
import os
if not os.path.basename(os.getcwd()) == "ElementsConcentration":
  os.chdir("ElementsConcentration")

from nutrients_regressor.chain_mlfunctions import *
from nutrients_regressor.data_processing import *
from nutrients_regressor.sp_functions import SpectralData


In [8]:
def run_chain_regressor(spdata, elementsdata, idsfun, element_to_predict, model_name = 'pls', chain = None):

    testrc = ElementsChainRegressor(base_estimator=set_model(model_name, nworkers = -1,scaler ='minmax'),
                                    spdata = spdata,
                                    elements_concentration = elementsdata,
                                    random_state=42,
                                    ids_order = idsfun)
    
    final_chain, performances = testrc.find_best_chain(element_to_predict,kfolds=10, not_include = ['Cd','K', 'B','Fe','Cu'], checkpoint_path = '/content/drive/MyDrive/PhD/elements_concentration/results_rcpaper',
                                                       chain = chain,
                                                       suffix_check = model_name)
    


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
idtestvalues = [240, 381, 147,  65, 677, 721,  24, 356, 424, 667, 732,  33, 483,
       669, 330,  29, 413, 559, 663, 716, 548,   8, 443, 671, 655, 363,
       675,  64, 111,   0, 430, 708, 278, 651, 262, 705, 197, 662, 137,
        10,  45, 472, 387, 455, 733, 157, 181, 495, 557, 238, 506, 294,
        61, 513, 544, 415, 234, 360,  13, 372, 643,  90, 600, 193,  16,
       344, 586, 396, 467,  75, 400, 567, 726, 342]

## folders
sp_path = "/content/drive/MyDrive/PhD/elements_concentration/spectral_data.csv"
nut_values_path = "/content/drive/MyDrive/PhD/elements_concentration/nutrient_values_withoutdate.csv"

## setting model and element to predict
element = "Mn"
modelname = "svr_linear"

## read hyperspectral data after being processed
spdata = SpectralData(sp_path, columns_suffix = 'WL.', apply_sg = True)
# get first derivative
spdata = spdata.first_derivative.reset_index().drop(['index'], axis = 1)

## read nutrient concentration values
elementsdata = ElementsData(nut_values_path)

## creating partitions
idsfun = SplitIds(spdata.shape[0],val_perc =None, test_perc = 10,seed = 123, shuffle = True, testids_fixed = idtestvalues)


print('*'*10, modelname)
print('*'*5, element)

## finding the best chain
if element == "Mn": # if you wnat to start from a initial chain
  run_chain_regressor(spdata, elementsdata, idsfun, element, model_name = modelname, chain = "Co_Mn")
else:
  run_chain_regressor(spdata, elementsdata, idsfun, element, model_name = modelname)

********** svr_linear
***** Mn
initial elements  ['B', 'Ca43', 'Cd', 'Cu', 'Fe', 'K', 'Li', 'Mg', 'Mo', 'Na', 'P', 'Rb', 'S', 'Sr', 'Zn']
